# Trabajo Práctico N° 1 - Parte 3
Integrantes:
- Martinez Brenda Carolina
- Fernandez Rocio Belen
- Turello Martina Gloria
- Forestiero Camila Julieta

## Enunciado
Dados los siguientes enunciados, elija un lenguaje (C++, Python y Java) para cada uno de ellos y
codifíquelos. Cabe aclarar que no se puede repetir el lenguaje entre enunciados.

a) Una empresa desea optimizar el espacio utilizado en el edificio donde opera, para esto planean
tener un solo baño con capacidad para tres (3) empleados, pero dado que actualmente su plantilla
de empleados se compone de hombres y mujeres, la empresa quiere garantizar que en el baño no se
encuentren hombres y mujeres al mismo tiempo.
Desarrolle un programa que implemente el funcionamiento de dicho baño utilizando algún método
de comunicación y/o sincronización.

b) En una reserva natural sudafricana existe un profundo cañón, los babuinos del lugar utilizan una
cuerda atada en ambos extremos para cruzar de un lado al otro. La cuerda es lo suficientemente
fuerte como para soportar a cinco (5) babuinos colgados al mismo tiempo, de superar esa cantidad
la cuerda se romperá. Por razones desconocidas si dos babuinos que van en direcciones opuestas se
encuentran en medio del cruce, estos se ponen a pelear hasta que alguno cae al vacío.
Desarrolle un programa utilizando algún método de comunicación y/o sincronización que represente
el cruce de los babuinos garantizando que ninguno muera.

c) En un supermercado existen dos (2) repositores que reponen los productos de una góndola con
capacidad máxima de diez (10) productos, dichos repositores trabajan con alternancia estricta, es
decir que una vez repone uno, luego el otro y así sucesivamente. También existen N clientes que van
tomando los productos de las góndolas (1 a 2 productos de manera aleatoria). Cuando la góndola se
vacía, el repositor repone la totalidad de productos. Cabe destacar que mientras se están reponiendo los productos, los clientes no pueden tomarlos, así como también si un cliente no obtuvo
la cantidad de productos que necesitaba, esperará hasta que la góndola se complete.
Desarrolle un programa utilizando algún método de comunicación y/o sincronización que represente
el funcionamiento anteriormente planteado para un número N de clientes donde N será pasado
como parámetro al programa.

##Elección de Lenguajes

###Ejercicio A: Baño de Empresa
Java fue elegido para este enunciado debido a que posee una gran capacidad para gestionar hilos y exclusión mutua con facilidad, superando a Python en rendimiento y siendo más sencillo que C++ para este tipo de problema.

###Ejercicio B: Cruce de Babuinos
C++ fue elegido para este enunciado ya que se necesita un control detallado y eficiente de la concurrencia y los recursos. Este lenguaje sobresale frente a Python, el cual se encuentra limitado por el Global Interpreter Lock (GIL), y a Java, ya que C++ permite una interacción más directa con el hardware, lo que garantiza un rendimiento óptimo.

###Ejercicio C: Reposición de Productos
Python fue seleccionado para este enunciado por su simplicidad y facilidad para manejar sincronización en escenarios donde no se requiere un paralelismo extremo. En comparación, ofrece mayor rapidez y facilidad de desarrollo que C++, y es más conciso que Java, lo que lo hace ideal para resolver problemas con menor complejidad técnica.

##Ejercicio A: Baño de Empresa

###Escritura del Código del Programa

In [3]:
%%writefile Bano.java
import java.util.concurrent.Semaphore;
import java.util.LinkedList;
import java.util.Queue;

class Bano
{
  private static int NoCapacidad = 0;
  private static int InicioContadorCola = 1;
  private static int BanoVacio = 0;

  private int capacidad = 3;
  private Semaphore semaforoCapacidad = new Semaphore(capacidad, true);
  private Semaphore semaforoGenero = new Semaphore(1, true);
  private String generoEnUso = "";
  private int contadorPersonaBano = 0;
  private Queue<Empleado> colaEspera = new LinkedList<>();

  public synchronized void entrarAlBano(Empleado empleado) throws InterruptedException
  {
    String nombre = empleado.getNombre();
    String genero = empleado.getGenero();

    System.out.println(nombre + " (" + genero + ") intenta entrar al baño");

    if (!colaEspera.contains(empleado))
    {
      colaEspera.add(empleado);
      System.out.println(nombre + " (" + genero + ") ha sido agregado a la cola de espera.");
      int numero = InicioContadorCola;
      for (Empleado emp : colaEspera)
      {
        System.out.println("\t[" + numero + "| " + emp.getNombre() + " (" + emp.getGenero() + ")]");
        numero++;
      }
    }

    while ((!generoEnUso.isEmpty() && !generoEnUso.equals(genero)) || semaforoCapacidad.availablePermits() == NoCapacidad || colaEspera.peek() != empleado)
    {
      wait();
    }

    colaEspera.remove();
    System.out.println(nombre + " (" + genero + ") ha sido removido de la cola de espera.");
    int numero = InicioContadorCola;
    for (Empleado emp : colaEspera)
    {
      System.out.println("\t[" + numero + "| " + emp.getNombre() + " (" + emp.getGenero() + ")]");
      numero++;
    }

    if (generoEnUso.isEmpty())
    {
      semaforoGenero.acquire();
      generoEnUso = genero;
      System.out.println(nombre + " (" + genero + ") ha entrado al baño. Solo puede haber " + genero + " hasta que salga.");
    }

    semaforoCapacidad.acquire();
    contadorPersonaBano++;
    System.out.println(nombre + " (" + genero + ") ha entrado al baño. Espacios restantes: " + semaforoCapacidad.availablePermits());
  }

  public synchronized void salirDelBano(String nombre, String genero)
  {
    semaforoCapacidad.release();
    contadorPersonaBano--;
    System.out.println(nombre + " (" + genero + ") ha salido del baño. Espacios restantes: " + semaforoCapacidad.availablePermits());

    if (contadorPersonaBano == BanoVacio)
    {
      generoEnUso = "";
      System.out.println("El baño está vacío. No hay nadie más en el baño. Ya se puede cambiar de género.");
      semaforoGenero.release();
    }

    notifyAll();
  }
}

class Empleado implements Runnable
{

  private static int tiempoEnBano = 1000;

  private Bano bano;
  private String nombre;
  private String genero;

  public Empleado(Bano bano, String nombre, String genero)
  {
    this.bano = bano;
    this.nombre = nombre;
    this.genero = genero;
  }

  public String getNombre()
  {
    return nombre;
  }

  public String getGenero()
  {
    return genero;
  }

  @Override
  public void run()
  {
    try
    {
      bano.entrarAlBano(this);
      Thread.sleep((int) (Math.random() * tiempoEnBano));
      bano.salirDelBano(nombre, genero);
    }
    catch (InterruptedException e)
    {
      e.printStackTrace();
    }
  }
}

Overwriting Bano.java


In [4]:
%%writefile Main.java
import java.util.ArrayList;
import java.util.List;
import java.util.Random;

public class Main
{

  private static int semilla = 5;

  public static void main(String[] args)
  {
    Bano bano = new Bano();

    List<String> nombresHombres = new ArrayList<>(List.of("Daro", "Matias", "Esteban", "Maximo", "Mariano", "Messi", "Duki", "Diego", "Victor", "Fran", "Pato"));
    List<String> nombresMujeres = new ArrayList<>(List.of("Martu", "Ro", "Bren", "Cami", "Shakira", "Pampita", "Nicky", "Luz", "Emilia"));

    Random random = new Random();
    int cantidadEmpleados = random.nextInt(semilla) + semilla;

    Thread[] empleados = new Thread[cantidadEmpleados];
    for (int i = 0; i < empleados.length; i++)
    {
      String genero = random.nextBoolean() ? "Hombre" : "Mujer";
      String nombre;

      if (genero.equals("Hombre") && !nombresHombres.isEmpty())
      {
        nombre = nombresHombres.remove(random.nextInt(nombresHombres.size()));
      }
      else if (genero.equals("Mujer") && !nombresMujeres.isEmpty())
      {
        nombre = nombresMujeres.remove(random.nextInt(nombresMujeres.size()));
      }
      else
      {
        System.out.println("No hay suficientes nombres disponibles.");
        break;
      }

      empleados[i] = new Thread(new Empleado(bano, nombre, genero));
      empleados[i].start();
    }

    for (Thread empleado : empleados)
    {
      try
      {
        empleado.join();
      }
      catch (InterruptedException e)
      {
        e.printStackTrace();
      }
    }
  }
}

Overwriting Main.java


###Compilación del Programa

In [5]:
!javac Bano.java
!javac Main.java

###Ejecución del Programa

In [6]:
!java Main

Maximo (Hombre) intenta entrar al baño
Maximo (Hombre) ha sido agregado a la cola de espera.
	[1| Maximo (Hombre)]
Maximo (Hombre) ha sido removido de la cola de espera.
Maximo (Hombre) ha entrado al baño. Solo puede haber Hombre hasta que salga.
Maximo (Hombre) ha entrado al baño. Espacios restantes: 2
Martu (Mujer) intenta entrar al baño
Martu (Mujer) ha sido agregado a la cola de espera.
	[1| Martu (Mujer)]
Pampita (Mujer) intenta entrar al baño
Pampita (Mujer) ha sido agregado a la cola de espera.
	[1| Martu (Mujer)]
	[2| Pampita (Mujer)]
Nicky (Mujer) intenta entrar al baño
Nicky (Mujer) ha sido agregado a la cola de espera.
	[1| Martu (Mujer)]
	[2| Pampita (Mujer)]
	[3| Nicky (Mujer)]
Emilia (Mujer) intenta entrar al baño
Emilia (Mujer) ha sido agregado a la cola de espera.
	[1| Martu (Mujer)]
	[2| Pampita (Mujer)]
	[3| Nicky (Mujer)]
	[4| Emilia (Mujer)]
Victor (Hombre) intenta entrar al baño
Victor (Hombre) ha sido agregado a la cola de espera.
	[1| Martu (Mujer)]
	[2| Pampita 

##Ejercicio B: Cruce de Babuinos

###Escritura del Código del Programa

In [10]:
%%writefile cruce.cpp
#include <iostream>
#include <thread>
#include <mutex>
#include <condition_variable>
#include <chrono>

const int MAX_BABOONS = 5;
const int TOTAL_BABOONS = 10;
const int LEFT_TO_RIGHT = 1;
const int RIGHT_TO_LEFT = 2;
const int MIN_CROSSING_TIME = 1;
const int MAX_CROSSING_TIME = 3;
const int DIRECTION_START_INDEX = 0;
const int DIRECTION_OFFSET = 1;
const int DIRECTION_MODIFIER = 2;
const int NO_CROSSING_BABOONS = 0;
const int OK = 0;

class Semaphore
{
  public:
    Semaphore(int count = NO_CROSSING_BABOONS) : count_(count) {}

    void acquire()
    {
      std::unique_lock<std::mutex> lock(mtx_);
      cv_.wait(lock, [this]()
        { return count_ > NO_CROSSING_BABOONS; });
      --count_;
    }

    void release()
    {
      std::unique_lock<std::mutex> lock(mtx_);
      ++count_;
      cv_.notify_one();
    }

  private:
    std::mutex mtx_;
    std::condition_variable cv_;
    int count_;
};

std::mutex mtx;
std::condition_variable cv;
Semaphore sem(MAX_BABOONS);
int crossing_baboons = 0;
int direction = 0;

std::mutex print_mtx;

void acquire_rope(int dir)
{
  std::unique_lock<std::mutex> lock(mtx);
  while (crossing_baboons > NO_CROSSING_BABOONS && direction != dir)
  {
    cv.wait(lock);
  }

  direction = dir;

  sem.acquire();

  crossing_baboons++;
}

void release_rope()
{
  std::unique_lock<std::mutex> lock(mtx);
  crossing_baboons--;
  if (crossing_baboons == NO_CROSSING_BABOONS)
  {
    direction = DIRECTION_START_INDEX;
    cv.notify_all();
  }

  sem.release();
}

void simulate_crossing(int id, int dir)
{

  std::lock_guard<std::mutex> lock(print_mtx);
  std::cout << "Babuino " << id << " está cruzando en dirección " << (dir == LEFT_TO_RIGHT ? "izquierda a derecha" : "derecha a izquierda") << std::endl;

  std::this_thread::sleep_for(std::chrono::seconds(MIN_CROSSING_TIME + rand() % MAX_CROSSING_TIME));

  std::cout << "Babuino " << id << " terminó de cruzar." << std::endl;
}

void baboon_cross(int id, int dir)
{
  acquire_rope(dir);
  simulate_crossing(id, dir);
  release_rope();
}

int main()
{
  srand(time(0));

  std::thread baboons[TOTAL_BABOONS];

  for (int i = DIRECTION_START_INDEX; i < TOTAL_BABOONS; i++)
  {
    int dir = (i % DIRECTION_MODIFIER) + DIRECTION_OFFSET;
    baboons[i] = std::thread(baboon_cross, i, dir);
  }

  for (int i = DIRECTION_START_INDEX; i < TOTAL_BABOONS; i++)
  {
    baboons[i].join();
  }

  return OK;
}

Overwriting cruce.cpp


###Compilación del Programa

In [11]:
!g++ -std=c++11 -o cruce cruce.cpp

###Ejecución del Programa

In [12]:
!./cruce

Babuino 1 está cruzando en dirección derecha a izquierda
Babuino 1 terminó de cruzar.
Babuino 3 está cruzando en dirección derecha a izquierda
Babuino 3 terminó de cruzar.
Babuino 9 está cruzando en dirección derecha a izquierda
Babuino 9 terminó de cruzar.
Babuino 7 está cruzando en dirección derecha a izquierda
Babuino 7 terminó de cruzar.
Babuino 5 está cruzando en dirección derecha a izquierda
Babuino 5 terminó de cruzar.
Babuino 8 está cruzando en dirección izquierda a derecha
Babuino 8 terminó de cruzar.
Babuino 0 está cruzando en dirección izquierda a derecha
Babuino 0 terminó de cruzar.
Babuino 6 está cruzando en dirección izquierda a derecha
Babuino 6 terminó de cruzar.
Babuino 4 está cruzando en dirección izquierda a derecha
Babuino 4 terminó de cruzar.
Babuino 2 está cruzando en dirección izquierda a derecha
Babuino 2 terminó de cruzar.


##Ejercicio C: Reposición de Productos

###Escritura del Código del Programa

In [15]:
%%writefile ejercicioC.py
import threading
import random
import time
import sys

ARG_CANT_CLIENTES = 1
CAPACIDAD_GONDOLA = 10
CANT_REPOSITORES = 2
TIEMPO_REPOSICION = 1
TIEMPO_TOMA_PRODUCTO = 1

productos_en_gondola = CAPACIDAD_GONDOLA
turno_repositor = 1
fin_supermercado = False

mutex_gondola = threading.Lock()
cond_gondola = threading.Condition(mutex_gondola)

def reponer_productos(nro_repositor):
  global productos_en_gondola, turno_repositor, fin_supermercado

  while True:
    with cond_gondola:
      while (productos_en_gondola > 0 or turno_repositor != nro_repositor) and not fin_supermercado:
        cond_gondola.wait()
      if fin_supermercado:
        break
      print(f"Repositor {nro_repositor} va a reponer la góndola")
      time.sleep(TIEMPO_REPOSICION)
      productos_en_gondola = CAPACIDAD_GONDOLA
      print("La góndola ha sido rellenada completamente.")
      turno_repositor = (turno_repositor % CANT_REPOSITORES) + 1
      cond_gondola.notify_all()

def comprar(nro_cliente):
  global productos_en_gondola

  print(f"Cliente {nro_cliente} ha ingresado a comprar.")
  productos_a_tomar = random.randint(1, 2)
  productos_tomados = 0

  while productos_tomados < productos_a_tomar:
    with cond_gondola:
      while productos_en_gondola == 0:
        print(f"Cliente {nro_cliente} detecta la góndola vacía y espera.")
        cond_gondola.notify_all()
        cond_gondola.wait()
      productos_en_gondola -= 1
      print(f"Cliente {nro_cliente} tomó un producto. Productos restantes: {productos_en_gondola}")
    productos_tomados += 1
    time.sleep(TIEMPO_TOMA_PRODUCTO)

  print(f"Cliente {nro_cliente} ha terminado su compra y se va.")

def supermercado(cantidad_de_clientes):
  global fin_supermercado

  hilo_repositor1 = threading.Thread(target=reponer_productos, args=(1,))
  hilo_repositor2 = threading.Thread(target=reponer_productos, args=(2,))
  hilo_repositor1.start()
  hilo_repositor2.start()

  hilos_clientes = [threading.Thread(target=comprar, args=(i,)) for i in range(1, cantidad_de_clientes + 1)]
  for h in hilos_clientes:
    h.start()

  for h in hilos_clientes:
    h.join()

  global fin_supermercado
  fin_supermercado = True
  with cond_gondola:
    cond_gondola.notify_all()

  hilo_repositor1.join()
  hilo_repositor2.join()
  print("Fin de la compra")

if __name__ == "__main__":
  cantidad_de_clientes = int(sys.argv[ARG_CANT_CLIENTES])
  supermercado(cantidad_de_clientes)

Overwriting ejercicioC.py


###Ejecución del Programa

In [17]:
!python3 ejercicioC.py 20

Cliente 1 ha ingresado a comprar.
Cliente 1 tomó un producto. Productos restantes: 9
Cliente 2 ha ingresado a comprar.
Cliente 2 tomó un producto. Productos restantes: 8
Cliente 3 ha ingresado a comprar.
Cliente 3 tomó un producto. Productos restantes: 7
Cliente 4 ha ingresado a comprar.
Cliente 4 tomó un producto. Productos restantes: 6
Cliente 5 ha ingresado a comprar.
Cliente 5 tomó un producto. Productos restantes: 5
Cliente 6 ha ingresado a comprar.
Cliente 6 tomó un producto. Productos restantes: 4
Cliente 7 ha ingresado a comprar.
Cliente 7 tomó un producto. Productos restantes: 3
Cliente 8 ha ingresado a comprar.
Cliente 8 tomó un producto. Productos restantes: 2
Cliente 9 ha ingresado a comprar.
Cliente 9 tomó un producto. Productos restantes: 1
Cliente 10 ha ingresado a comprar.
Cliente 10 tomó un producto. Productos restantes: 0
Cliente 11 ha ingresado a comprar.
Cliente 11 detecta la góndola vacía y espera.
Cliente 12 ha ingresado a comprar.
Cliente 12 detecta la góndola va

#Conclusiones

El desarrollo de este trabajo práctico permitió una valiosa comparación entre los lenguajes C++, Java y Python en el contexto de problemas de comunicación y sincronización. Se trató de una excelente oportunidad para comparar los tres lenguajes, analizando cuál era la mejor opción en cada uno de los tres ejercicios planteados, pertenecientes a distintos contextos y buscando diferentes soluciones.

En el ejercicio del baño de la empresa, realizado con Java, enfrentamos desafíos relacionados con la gestión de acceso al baño, particularmente en lo que respecta a la correcta implementación de una cola de personas y la lógica de género. Inicialmente, el género que ingresaba primero monopolizaba el acceso al baño, lo que nos obligó a reformular la lógica para que se respetara el orden de la fila, independientemente del género. También sucedía que no se respetaba el orden de aparición de la persona, por lo que las personas ingresaban al baño de forma aleatoria, motivo que resultó en la implementación de una cola de personas.

En el ejercicio del cruce de babuinos, desarrollado con C++, la implementación de un mutex fue clave para evitar la desorganización en los mensajes de anuncio de los babuinos cruzando y finalizando su cruce, lo que nos permitió sincronizar correctamente los hilos y mejorar la claridad en la visualización de los resultados.

En el ejercicio de reposición de productos, implementado con Python, al intentar implementar la solución con semáforos, enfrentamos dificultades para sincronizar adecuadamente el acceso a la góndola y gestionar las señales de reposición. Un problema recurrente fue que, si varios clientes llegaban mientras la góndola estaba vacía, se enviaban múltiples señales de reposición, lo que provocaba una sobrecarga innecesaria. Además, en algunas situaciones, un cliente podía tomar productos mientras se estaba realizando la reposición, lo cual es incorrecto según las especificaciones del enunciado. Finalmente, resolvimos estos problemas investigando e implementando diversas herramientas proporcionadas por el lenguaje y sus bibliotecas, logrando una sincronización eficiente y un comportamiento correcto.